**1. LLMChain**

The LLMChain is the most basic and common chain. It combines a PromptTemplate and an LLM. It takes an input, formats it with the prompt template, passes it to the LLM, and returns the LLM's output.

Structure: Input -> PromptTemplate -> LLM -> Output

In [3]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'


In [6]:
# Initialize the LLM
llm = OpenAI(temperature=0.7)

In [5]:
# Create a prompt template
prompt = PromptTemplate(
    input_variables=["name", "mood"],
    template="Write a short greeting for someone named {name} who is feeling {mood}."
)


In [7]:

# Create the chain
greeting_chain = LLMChain(llm=llm, prompt=prompt)


/tmp/ipython-input-3487607905.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  greeting_chain = LLMChain(llm=llm, prompt=prompt)


In [8]:

# Run the chain
result = greeting_chain.invoke({"name": "Alice", "mood": "joyful"})
print(result['text'])



"Hello Alice, it's wonderful to see you radiating with joy! Keep spreading that positive energy and have a fantastic day!"


In [9]:
# Product Description Generator:

prompt = PromptTemplate(
    input_variables=["product", "audience"],
    template="Write a one-sentence description of {product} for {audience}."
)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke({"product": "a smartwatch", "audience": "athletes"})

print(result['text'])




A high-tech wearable device designed to track and optimize athletic performance and fitness levels.


In [10]:
#Example

from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()

ITfirm_template = """
I want you to act as a consultant for a new IT firm.
Return a list of firm names and a locality which is suitable for such a firm in India.
The name should be catchy and the locality should yield profit.
What are some good names and respective area for IT firm that does {firm_description}
"""

prompt_template = PromptTemplate(
    input_variables=["firm_description"],
    template=ITfirm_template,
)

description = "It is software dev firm specifically focusing on automation software"

prompt_template.format(firm_description=description)

chain = LLMChain(llm=llm, prompt=prompt_template)

print(chain.run(description))

/tmp/ipython-input-2138360038.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run(description))


1. TechAutomate Solutions - Bangalore, Karnataka
2. InnovateSoft Automation - Pune, Maharashtra
3. CodeCraft Automation - Hyderabad, Telangana
4. SmartTech Automations - Gurgaon, Haryana
5. AutomateIT Technologies - Chennai, Tamil Nadu


# **Sequential Chains**

A SequentialChain is used to execute multiple chains in a predefined order, passing the output of one chain as input to the next. This is the formal, preferred way to create multi-step workflows .

There are two main types:

SimpleSequentialChain: For chains with a single input and single output.

SequentialChain: For chains with multiple inputs and/or multiple outputs.

In [14]:
from langchain.chains import SimpleSequentialChain


In [17]:
llm = ChatOpenAI(model_name='gpt-4o-mini')

In [18]:

# First Chain: Translates English to French
translation_prompt = PromptTemplate(
    input_variables=["text"],
    template="Translate this English text to French: {text}"
)
translate_chain = LLMChain(llm=llm, prompt=translation_prompt)


In [19]:
# Second Chain: Summarizes the translated text
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this French text in one sentence: {text}"
)
summarize_chain = LLMChain(llm=llm, prompt=summary_prompt)



In [21]:
# Combine them
overall_chain = SimpleSequentialChain(chains=[translate_chain, summarize_chain], verbose=True) #, verbose=True


In [22]:
final_result = overall_chain.run("A long article about the history of the internet...")




> Entering new SimpleSequentialChain chain...
Un long article sur l'histoire d'Internet...
Un article détaillé retrace l'évolution et les événements marquants de l'histoire d'Internet.

> Finished chain.


In [23]:
print(final_result)

Un article détaillé retrace l'évolution et les événements marquants de l'histoire d'Internet.


Generate a Story and its Title (SequentialChain):

This uses SequentialChain because we have two final outputs we want to keep.

In [24]:
from langchain.chains import SequentialChain

In [25]:
# Chain 1: Generate a story idea
idea_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Generate a short story idea in the {genre} genre."
)
idea_chain = LLMChain(llm=llm, prompt=idea_prompt, output_key="story_idea")

In [26]:
# Chain 2: Generate a title based on the story idea
title_prompt = PromptTemplate(
    input_variables=["story_idea"],
    template="Create a catchy title for this story idea: {story_idea}"
)
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [27]:
# Combine them, defining input and output variables
overall_chain = SequentialChain(
    chains=[idea_chain, title_chain],
    input_variables=["genre"], # The initial input
    output_variables=["story_idea", "title"], # The final outputs we want
    verbose=True
)


In [28]:
result = overall_chain.invoke({"genre": "sci-fi"})




> Entering new SequentialChain chain...

> Finished chain.


In [29]:
print(f"Idea: {result['story_idea']}")


Idea: **Title: "Echoes of Titan"**

**Synopsis:** In the year 2147, humanity has established a thriving underwater colony on Titan, Saturn's largest moon, where researchers have discovered an ancient alien artifact that emits mysterious signals. The story follows Dr. Elara Voss, a brilliant xenobiologist, who is leading a team to decode these signals, believing they hold the key to unlocking advanced technologies and insights into pre-human civilizations.

As Elara and her team delve deeper into the artifact's secrets, they begin to experience strange phenomena: vivid dreams of a lush, green world, cryptic messages in their minds, and glimpses of a civilization that thrived eons ago. As the lines between reality and memory blur, the team uncovers that the artifact is not just a relic—it is a sentient device that connects to the consciousness of living beings.

When corporate interests threaten to exploit the artifact for profit, Elara must rally her team to protect it, uncovering its t

In [30]:
print(f"Title: {result['title']}")

Title: **Title: "Divine Signals: The Titan Awakening"**


**RouterChain**

A RouterChain is an advanced chain that decides which of several specialized sub-chains to use based on the input. It's like a "switchboard" for your chains.

Structure: Input -> RouterChain -> (Chooses best sub-chain) -> Sub-chain processes input -> Output

In [33]:
from langchain.chains import LLMRouterChain, RouterChain
from langchain.chains.router import MultiPromptChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [34]:
# 1. Initialize your LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [35]:
# 2. Define destination prompts (each acts like a "sub-chain")
math_prompt = PromptTemplate(
    template="You are a math assistant. Solve this problem: {input}",
    input_variables=["input"],
)

In [36]:
history_prompt = PromptTemplate(
    template="You are a historian. Answer this question: {input}",
    input_variables=["input"],
)

In [37]:
# 3. Put them into a dictionary
destination_chains = {
    "math": math_prompt | llm,
    "history": history_prompt | llm,
}

In [38]:
# 4. Define a default chain (if no route matches)
default_chain = llm

In [40]:
# 2. Prepare prompt_infos with required fields
prompt_infos = [
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": "You are a math expert. Solve: {input}",
    },
    {
        "name": "history",
        "description": "Good for answering history questions",
        "prompt_template": "You are a historian. Answer: {input}",
    },
]

In [44]:
# 3. Create the MultiPromptChain
chain = MultiPromptChain.from_prompts(
    llm=llm,
    prompt_infos=prompt_infos,
    default_chain=None,
    verbose=True,
)

# 4. Run it
result = chain.run("What is 12 × 8?")
print(result)



> Entering new MultiPromptChain chain...
math: {'input': 'What is 12 × 8?'}
> Finished chain.
12 × 8 = 96
